In [2]:
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse

import util

In [3]:
def extract_feats(ffs, direc="train", global_feat_dict=None):
    """
    arguments:
      ffs are a list of feature-functions.
      direc is a directory containing xml files (expected to be train or test).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.

    returns: 
      a sparse design matrix, a dict mapping features to column-numbers,
      a vector of target classes, and a list of system-call-history ids in order 
      of their rows in the design matrix.
      
      Note: the vector of target classes returned will contain the true indices of the
      target classes on the training data, but will contain only -1's on the test
      data
    """
    fds = [] # list of feature dicts
    classes = []
    ids = [] 
    for datafile in os.listdir(direc):
        # extract id and true class (if available) from filename
        id_str,clazz = datafile.split('.')[:2]
        ids.append(id_str)
        # add target class if this is training data
        try:
            classes.append(util.malware_classes.index(clazz))
        except ValueError:
            # we should only fail to find the label in our list of malware classes
            # if this is test data, which always has an "X" label
            assert clazz == "X"
            classes.append(-1)
        rowfd = {}
        # parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
        # accumulate features
        [rowfd.update(ff(tree)) for ff in ffs]
        fds.append(rowfd)
    
    X,feat_dict = make_design_mat(fds,global_feat_dict)
    return X, feat_dict, np.array(classes), ids

In [4]:
def make_design_mat(fds, global_feat_dict=None):
    """
    arguments:
      fds is a list of feature dicts (one for each row).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.
       
    returns: 
        a sparse NxD design matrix, where N == len(fds) and D is the number of
        the union of features defined in any of the fds 
    """
    if global_feat_dict is None:
        all_feats = set()
        [all_feats.update(fd.keys()) for fd in fds]
        feat_dict = dict([(feat, i) for i, feat in enumerate(sorted(all_feats))])
    else:
        feat_dict = global_feat_dict
    
    cols = []
    rows = []
    data = []        
    for i in xrange(len(fds)):
        temp_cols = []
        temp_data = []
        for feat,val in fds[i].iteritems():
            try:
                # update temp_cols iff update temp_data
                temp_cols.append(feat_dict[feat])
                temp_data.append(val)
            except KeyError as ex:
                if global_feat_dict is not None:
                    pass  # new feature in test data; nbd
                else:
                    raise ex

        # all fd's features in the same row
        k = len(temp_cols)
        cols.extend(temp_cols)
        data.extend(temp_data)
        rows.extend([i]*k)

    assert len(cols) == len(rows) and len(rows) == len(data)
   

    X = sparse.csr_matrix((np.array(data),
                   (np.array(rows), np.array(cols))),
                   shape=(len(fds), len(feat_dict)))
    return X, feat_dict

In [5]:
def system_call_count_feats(tree):
    """
    arguments:
      tree is an xml.etree.ElementTree object
    returns:
      a dictionary mapping 'num_system_calls' to the number of system_calls
      made by an executable (summed over all processes)
    """
    c = Counter()
    in_all_section = False
    for el in tree.iter():
        if el.tag == "load_dll":
            print el.attrib["filename_hash"]
        # ignore everything outside the "all_section" element
        if el.tag == "all_section" and not in_all_section:
            in_all_section = True
        elif el.tag == "all_section" and in_all_section:
            in_all_section = False
        elif in_all_section:
            c['num_system_calls'] += 1
    return c

In [6]:
#############################
#
#  Below is main function
#
#############################

In [7]:
train_dir = "exp"
test_dir = "../test"
outputfile = "experiment_predictions.csv"  # feel free to change this or take it as an argument

In [37]:
# TODO put the names of the feature functions you've defined above in this list
ffs = [system_call_count_feats]

In [38]:
# extract features
print "extracting training features..."
X_train,global_feat_dict,t_train,train_ids = extract_feats(ffs, train_dir)

extracting training features...
e753d19a2e3b98b2b3b8f02f276092096d10f22d
c88d57cc99f75cd928b47b6e444231f26670138f
08fe9ff1fe9b8fd237adedb10d65fb0447b91fe5
0f37018f672c7635691f7317ade3c5a63904ec96
ea2e9bac1789b53d7efcd675a63f4a2b44898439
f3bb6474ec18ee9cccc02ef44510a78b42058a41
31166d1e35a911a082149f5efac2dac228109615
3168a173d177f470928b468db777640861dcb32e
70d5f97088cc9348bb9d10098af0738a696b96de
c7753525d5276ab92a01a8a3c39cfb5269da7f98
f9be7678080084bc3cb48d0cb64bba9ba98800f1
8ae378a352c4a13516bf1fe5a8615d8893d2ff2d
59cb82f432d529da3e81b030b2365c22a95fd35a
e753d19a2e3b98b2b3b8f02f276092096d10f22d
c88d57cc99f75cd928b47b6e444231f26670138f
ea2e9bac1789b53d7efcd675a63f4a2b44898439
70d5f97088cc9348bb9d10098af0738a696b96de


In [113]:
y_train = np.zeros((len(t_train),len(util.malware_classes)))
y_train[np.arange(len(t_train)), t_train] = 1
y_train.shape

(3086, 15)

In [114]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(class_weight = "balanced", n_jobs = -1)
RF.fit(X_train, y_train)
print RF.n_features_
print RF.feature_importances_

2137
[0. 0. 0. ... 0. 0. 0.]


In [115]:
# import operator
# sorted_global_feat_dict = sorted(global_feat_dict.items(), key=operator.itemgetter(1))
# print sorted_global_feat_dict

In [116]:
RF.score(X_train, y_train)

0.9585223590408296

In [76]:
# get rid of training data and load test data
# del X_train
# del t_train
# del train_ids

In [77]:
print "extracting test features..."
X_test,_,t_ignore,test_ids = extract_feats(ffs, test_dir, global_feat_dict=global_feat_dict)

extracting test features...


In [78]:
preds = RF.predict(X_test)

In [79]:
preds[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [80]:
# TODO make predictions on text data and write them out
print "making predictions..."
results = np.argmax(preds, axis=1)

print "writing predictions..."
util.write_predictions(results, test_ids, outputfile)

making predictions...
writing predictions...


In [81]:
import csv 
import sys 

def reorder_submission(file_to_reorder, newfile_name = "experiment_results.csv"):
    # READ IN KEYS IN CORRECT ORDER AS LIST
    with open('keys.csv','r') as f:
        keyreader = csv.reader(f)
        keys = [key[0] for key in keyreader]

    # READ IN ALL PREDICTIONS, REGARDLESS OF ORDER
    with open(file_to_reorder) as f:
        oldfile_reader = csv.reader(f)
        D = {}
        for i,row in enumerate(oldfile_reader):
            if i == 0:
                continue
            _id, pred = row 
            D[_id] = pred

    # WRITE PREDICTIONS IN NEW ORDER
    with open(newfile_name,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('Id','Prediction'))
        for key in keys:
            writer.writerow((key,D[key]))

    print("".join(["Reordered ", file_to_reorder," and wrote to ", newfile_name]))

In [84]:
reorder_submission(outputfile, "experiment_randomforest_results_4.csv")

Reordered experiment_predictions.csv and wrote to experiment_randomforest_results_4.csv
